In [52]:
import pandas as pd
import numpy as np

In [53]:
data = pd.read_csv('../data/Vehicles.csv',encoding='utf-8')

# General info

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460437 entries, 0 to 460436
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CRASH_UNIT_ID        460437 non-null  int64  
 1   RD_NO                460437 non-null  object 
 2   CRASH_DATE           460437 non-null  object 
 3   UNIT_NO              460437 non-null  int64  
 4   UNIT_TYPE            460436 non-null  object 
 5   VEHICLE_ID           450064 non-null  float64
 6   MAKE                 450064 non-null  object 
 7   MODEL                448651 non-null  object 
 8   LIC_PLATE_STATE      410808 non-null  object 
 9   VEHICLE_YEAR         371782 non-null  float64
 10  VEHICLE_DEFECT       450064 non-null  object 
 11  VEHICLE_TYPE         450064 non-null  object 
 12  VEHICLE_USE          450064 non-null  object 
 13  TRAVEL_DIRECTION     450064 non-null  object 
 14  MANEUVER             450064 non-null  object 
 15  OCCUPANT_CNT     

# Missing and Unique values

In [55]:
for col in data.columns:
    print(str(col) +": "+ str(data[col].nunique()))

print("------------------------")
for col in data.columns:
    print(str(col) +": "+ str(data[col].isna().sum()))

CRASH_UNIT_ID: 460437
RD_NO: 257925
CRASH_DATE: 166465
UNIT_NO: 11
UNIT_TYPE: 7
VEHICLE_ID: 450064
MAKE: 665
MODEL: 1574
LIC_PLATE_STATE: 52
VEHICLE_YEAR: 131
VEHICLE_DEFECT: 17
VEHICLE_TYPE: 18
VEHICLE_USE: 25
TRAVEL_DIRECTION: 9
MANEUVER: 27
OCCUPANT_CNT: 40
FIRST_CONTACT_POINT: 14
------------------------
CRASH_UNIT_ID: 0
RD_NO: 0
CRASH_DATE: 0
UNIT_NO: 0
UNIT_TYPE: 1
VEHICLE_ID: 10373
MAKE: 10373
MODEL: 11786
LIC_PLATE_STATE: 49629
VEHICLE_YEAR: 88655
VEHICLE_DEFECT: 10373
VEHICLE_TYPE: 10373
VEHICLE_USE: 10373
TRAVEL_DIRECTION: 10373
MANEUVER: 10373
OCCUPANT_CNT: 10373
FIRST_CONTACT_POINT: 10388


# Changes to make in the attribute values

In [56]:
print("UNIT_NO:"+str(data["UNIT_NO"].unique())) #number of units implicated in the rd_no case
print("---------------")
print("UNIT_TYPE:"+str(data["UNIT_TYPE"].unique())) #Just nan
print("---------------")
print("LIC_PLATE_STATE:"+str(data["LIC_PLATE_STATE"].unique())) #we have xx state and also nan, 
print("---------------")
#lic plate state we should rename xx and nan as unknown
#MAKE AND TYPE IF NA APPARENTLY THEY ARE NOT RELEVANT SO WE CAN LEAVE THEM AS THEY ARE RIGHT NOW
print("VEHICLE_YEAR:"+str(data["VEHICLE_YEAR"].unique())) #TRY CHANGING THE WAY PANDAS IS HANDLING THE DATES
print("---------------")
#dates are badly handled
print("VEHICLE_DEFECT:"+str(data["VEHICLE_DEFECT"].unique())) #modify nan to unknown
print("---------------")
print("VEHICLE_TYPE:"+str(data["VEHICLE_TYPE"].unique())) #Change nan to UNKNOWN/NA
print("---------------")
print("VEHICLE_USE:"+str(data["VEHICLE_USE"].unique())) #Change nan to UNKNOWN/NA(same as above)
print("---------------")
print("TRAVEL_DIRECTION:"+str(data["TRAVEL_DIRECTION"].unique())) #nan to UNKNOWN
print("---------------")
print("MANEUVER:"+str(data["MANEUVER"].unique())) #nan to UNKNOWN/NA
print("---------------")
print("OCCUPANT_CNT:"+str(data["OCCUPANT_CNT"].unique())) #pay atention to these numbers they can be with bad
#input, and maybe is an int but is picked as something else, cause of 60 occupants, also nan leave it as nan
#as to not change the numerical attribute.
print("---------------")
print("FIRST_CONTACT_POINT:"+str(data["FIRST_CONTACT_POINT"].unique())) #nan to UNKNOWN
print("---------------")

UNIT_NO:[ 1  2  3  4  5  6  7  8  0  9 10]
---------------
UNIT_TYPE:['DRIVER' 'PARKED' 'PEDESTRIAN' 'BICYCLE' 'NON-MOTOR VEHICLE' 'DRIVERLESS'
 'NON-CONTACT VEHICLE' nan]
---------------
LIC_PLATE_STATE:['XX' 'IL' nan 'WI' 'IA' 'IN' 'MI' 'CA' 'TX' 'MN' 'OK' 'FL' 'OH' 'MO' 'ID'
 'KY' 'MS' 'AZ' 'ND' 'TN' 'GA' 'PA' 'MA' 'CO' 'WA' 'NC' 'KS' 'VA' 'NY'
 'OR' 'NV' 'NB' 'ME' 'RI' 'MD' 'NM' 'LA' 'SC' 'DE' 'AR' 'SD' 'NJ' 'CT'
 'WV' 'UT' 'AL' 'MT' 'NH' 'HI' 'DC' 'WY' 'AK' 'VT']
---------------
VEHICLE_YEAR:[  nan 2003. 2001. 2013. 2016. 2015. 2007. 2002. 2011. 2010. 2014. 2017.
 1999. 2006. 2004. 2012. 2008. 2009. 2000. 2018. 1997. 2005. 1995. 1998.
 2020. 2103. 1991. 2019. 1996. 1993. 1992. 1994. 1990. 9999. 1989. 1984.
 2204. 2915. 1986. 1900. 2913. 1988. 1987. 2205. 3007. 1982. 2107. 2041.
 2999. 2917. 1976. 2105. 2201. 1973. 2104. 2918. 1983. 1981. 1985. 2911.
 2916. 1980. 2099. 2208. 1979. 3017. 2033. 2207. 1978. 1901. 1970. 1969.
 2028. 1977. 2022. 2096. 1971. 2206. 1968. 1967. 1941. 2102.

# Changes to make in VEHICLE YEAR 

## a lot of them can be replaced in some way, the 9999 appears as a typo for 1999 since is the one that has a lot of occurences(257), the rest have at max 7-8 occurences. (CHECK ON THE WEB PAGE)

In [95]:
filtered_df = data[(data['VEHICLE_YEAR'] > 2025) & data['VEHICLE_YEAR'].notnull()]
print("total entries with wrong year date: " + str(len(filtered_df)))
filtered_df = filtered_df["VEHICLE_YEAR"].unique()
print("unique entries with wrong year date: " + str(filtered_df.size))
""" for i in filtered_df:
    print(str(i)+": "+str(data[data["VEHICLE_YEAR"] == i]["VEHICLE_YEAR"].count()))
 """




total entries with wrong year date: 422
unique entries with wrong year date: 61


' for i in filtered_df:\n    print(str(i)+": "+str(data[data["VEHICLE_YEAR"] == i]["VEHICLE_YEAR"].count()))\n '

# OCCUPANT COUNT

In [101]:
filtered_df = data[(data['OCCUPANT_CNT'] > 7) & data['OCCUPANT_CNT'].notnull()]
print("total entries with more than usual ppl: " + str(len(filtered_df)))
filtered_df = filtered_df["OCCUPANT_CNT"].unique()
print("unique entries with more than usual ppl: " + str(filtered_df.size))
""" for i in filtered_df:
    print(str(i)+": "+str(data[data["OCCUPANT_CNT"] == i]["OCCUPANT_CNT"].count())) """


total entries with more than usual ppl: 278
unique entries with more than usual ppl: 32


' for i in filtered_df:\n    print(str(i)+": "+str(data[data["OCCUPANT_CNT"] == i]["OCCUPANT_CNT"].count())) '

In [102]:
""" filtered = data[data["VEHICLE_YEAR"] == 2201]
filtered.count() """


' filtered = data[data["VEHICLE_YEAR"] == 2201]\nfiltered.count() '

In [58]:
data

,CRASH_UNIT_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,VEHICLE_ID,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT
0,561555,JC113649,01/12/2019 12:01:00 AM,1,DRIVER,535742.0,UNKNOWN,UNKNOWN,XX,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,S,STRAIGHT AHEAD,1.0,REAR-RIGHT
1,561563,JC113627,01/11/2019 11:36:00 PM,2,DRIVER,535738.0,"TOYOTA MOTOR COMPANY, LTD.",Highlander(beginning vehicle year 2001),IL,2003.0,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,S,STRAIGHT AHEAD,1.0,FRONT-RIGHT
2,561564,JC113627,01/11/2019 11:36:00 PM,1,DRIVER,535741.0,FORD,EXPLORER,IL,2001.0,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,E,STRAIGHT AHEAD,1.0,FRONT-LEFT
3,561540,JC113637,01/11/2019 11:31:00 PM,1,DRIVER,535714.0,CHEVROLET,MALIBU (CHEVELLE),IL,2013.0,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,SIDE-LEFT
4,561541,JC113637,01/11/2019 11:31:00 PM,2,DRIVER,535718.0,JEEP,LAREDO,IL,2016.0,NONE,PASSENGER,PERSONAL,S,STRAIGHT AHEAD,1.0,SIDE-LEFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460432,30751,HZ164689,02/24/2014 07:45:00 PM,2,DRIVER,29701.0,CHEVROLET,UNKNOWN,TN,2016.0,NONE,PASSENGER,PERSONAL,S,TURNING LEFT,1.0,REAR
460433,24495,HZ122950,01/21/2014 07:40:00 AM,1,DRIVER,23633.0,"TOYOTA MOTOR COMPANY, LTD.",COROLLA,IL,2005.0,NONE,PASSENGER,NOT IN USE,S,STRAIGHT AHEAD,1.0,SIDE-LEFT
460434,24496,HZ122950,01/21/2014 07:40:00 AM,2,DRIVER,23634.0,NISSAN,ROGUE,IL,2013.0,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,FRONT
460435,481321,JB442550,01/18/2014 06:14:00 PM,1,DRIVER,460655.0,MERCEDES-BENZ,UNKNOWN,IL,2016.0,UNKNOWN,PASSENGER,UNKNOWN/NA,E,LEAVING TRAFFIC LANE TO PARK,1.0,FRONT-RIGHT
